In [1]:
!which python

/home/cloucera/projects/pyholird/.venv/bin/python


In [28]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
import shap
import pathlib
import matplotlib.pyplot as plt
import dotenv
import os
from sklearn.model_selection import (
    ShuffleSplit,
    train_test_split,
)
import sys

plt.style.use("ggplot")

dotenv_filepath = pathlib.Path(dotenv.find_dotenv())
dotenv.load_dotenv(dotenv_filepath)
project_path = dotenv_filepath.parent


In [ ]:
sys.path.append(project_path.as_posix())

In [17]:
from src.datasets import get_disease_data

disease_name = "Acute_promyelocytic_leukemia"
experiment_fname =  "experiment.env"
disease_fpath = project_path.joinpath("notebooks", "data", "PanRD_ML", "results", disease_name, experiment_fname)

In [19]:
X, Y,  _, _ = get_disease_data(disease_fpath)

In [20]:
X.head()

,18,19,25,30,41,43,55,87,100,124,...,127833,134864,154807,162514,200959,203068,255738,338442,338557,548596
index,,,,,,,,,,,,,,,,,,,,,
GTEX.1117F.0226.SM.5GZZ7,0.317665,0.458580,0.475477,0.359000,0.189063,0.223048,0.153920,0.469224,0.317083,0.223048,...,0.124269,1.627636e-02,0.375524,0.248760,1.688435e-16,0.477527,0.133313,0.147872,0.143895,1.603941e-01
GTEX.111CU.1826.SM.5GZYN,0.310635,0.403279,0.450367,0.377898,0.202662,0.198975,0.210150,0.457888,0.357860,0.213907,...,0.132201,1.688435e-16,0.446102,0.220727,1.688435e-16,0.462783,0.045873,0.338529,0.281507,1.688435e-16
GTEX.111FC.0226.SM.5N9B8,0.273036,0.400844,0.439983,0.355321,0.243211,0.227091,0.118643,0.471758,0.343834,0.162724,...,0.167754,1.688435e-16,0.375780,0.241828,1.688435e-16,0.482021,0.079564,0.179598,0.160711,2.100090e-02
GTEX.111VG.2326.SM.5N9BK,0.304227,0.394599,0.441530,0.382837,0.224171,0.228116,0.161616,0.477043,0.338001,0.171134,...,0.135484,1.688435e-16,0.402339,0.287238,1.688435e-16,0.490011,0.169080,0.268850,0.250149,4.095755e-02
GTEX.111YS.2426.SM.5GZZQ,0.317121,0.469217,0.443515,0.370553,0.164537,0.166544,0.164537,0.457146,0.312981,0.188892,...,0.180401,1.688435e-16,0.425451,0.204232,1.688435e-16,0.463205,0.064560,0.250160,0.242333,1.743191e-02


In [22]:
Y.head()

,P.hsa04012.29,P.hsa04012.8,P.hsa04062.43,P.hsa04066.63,P.hsa04066.64,P.hsa04066.49,P.hsa04066.50,P.hsa04066.51,P.hsa04066.5,P.hsa04066.52,...,P.hsa04920.2,P.hsa04920.28,P.hsa04920.10,P.hsa04920.9,P.hsa04920.6,P.hsa04920.37,P.hsa04920.31,P.hsa04920.26,P.hsa04920.16,P.hsa04920.40
index,,,,,,,,,,,,,,,,,,,,,
GTEX.1117F.0226.SM.5GZZ7,0.252374,0.054407,0.027943,0.029371,0.028144,0.004076,0.017938,0.024551,0.030644,0.024258,...,0.065738,0.012099,0.008271,0.006859,0.000509,0.000044,8.639465e-06,2.070583e-05,0.000089,0.000430
GTEX.111CU.1826.SM.5GZYN,0.230129,0.055629,0.031996,0.026657,0.026461,0.003187,0.021076,0.023447,0.025674,0.022483,...,0.083765,0.013340,0.006220,0.004804,0.000734,0.000056,6.669274e-06,7.256976e-20,0.000162,0.000296
GTEX.111FC.0226.SM.5N9B8,0.209267,0.045224,0.026725,0.026258,0.023930,0.004435,0.013841,0.020312,0.023513,0.023102,...,0.055433,0.008503,0.008361,0.007435,0.000528,0.000047,8.947103e-20,8.947103e-20,0.000096,0.000375
GTEX.111VG.2326.SM.5N9BK,0.220617,0.033354,0.035292,0.031075,0.025414,0.004399,0.022378,0.033058,0.026103,0.022573,...,0.067624,0.009995,0.009119,0.006884,0.000343,0.000017,7.710451e-06,1.389914e-05,0.000085,0.000375
GTEX.111YS.2426.SM.5GZZQ,0.246832,0.058853,0.024007,0.024464,0.025996,0.001904,0.024972,0.024166,0.028324,0.026786,...,0.092475,0.014943,0.004791,0.003434,0.001175,0.000059,4.873428e-20,4.873428e-20,0.000303,0.000162


In [24]:
n = 100
cv = ShuffleSplit(n_splits=n, train_size=0.5, random_state=0)
cv = list(cv.split(X, Y))
cv = [(*train_test_split(cv[i][0], test_size=0.3), cv[i][1]) for i in range(n)]

In [38]:
i = 0
split = cv[i]
learn, val, test = split
X_learn = X.iloc[learn, :]
Y_learn = Y.iloc[learn, :]
X_val = X.iloc[val, :]
Y_val = Y.iloc[val, :]
X_test = X.iloc[test, :]
Y_test = Y.iloc[test, :]
X_train = pd.concat((X_learn, X_val), axis=0)
Y_train = pd.concat((Y_learn, Y_val), axis=0)

X.shape, Y.shape, X_learn.shape, Y_learn.shape, X_val.shape, Y_val.shape

((11688, 683),
 (11688, 105),
 (4090, 683),
 (4090, 105),
 (1754, 683),
 (1754, 105))

In [39]:
from src.models import get_model
n_features = X.shape[1]
model = get_model(n_features, n_jobs=24, debug=False)

In [40]:
model.fit(X_learn, Y_learn)

RandomForestRegressor(max_depth=8, max_features=46, n_estimators=200, n_jobs=24)

In [41]:
explainer = shap.GPUTreeExplainer(model, X_learn)

In [42]:
sv = explainer.shap_values(X_val)